In [1]:
import numpy as np
import gym

gym environment

In [2]:
env_name = 'FrozenLake-v0'

In [4]:
env = gym.make(env_name)

behavior define

In [ ]:
# active function: sigmoid
def sigmoid(v):
    return 1.0 / (1.0 + np.exp(-v))
vec_sigmoid = np.vectorize(sigmoid)

In [ ]:
# nn model define by layers:
# (node_count, active_function)
model_define = [
    (100, None),
    (env.action_space.n, vec_sigmoid),
]

In [ ]:
# binary expand categorical type to preprocess states
def binary_expand(n, idx=env.observation_space.n):
    if type(idx) is int:
        idx = np.array(list(range(5)))
    res = np.zeros(idx.shape)
    res[n==idx] = 1
    return res

preprocess_func = binary_expand

hyperparameters

In [3]:
episodes = 1000
learning_rate = 0.1
discount = 0.99

epsilon = 1
epsilon_decay = 0.99
epsilon_min = 0.01

deep Q engine

In [ ]:
model = []
prev_node_count = env.action_space.sample()
for node_count, activation_func in model_define:
    model.append((np.random.rand(prev_node_count, node_count), activation_func))
    prev_node_count = node_count

In [ ]:
# buffer to temporarily store hidden layer result for back propagation
hidden_layer_res_buf = []

In [ ]:
# preprocess state information into input parameters
preprocess_state = preprocess_func

In [ ]:
# forward pass, calculate predict value with current model
def model_forward(state):
    cur_res = state
    hidden_layer_res_buf.clear()
    for layer, activation_func in model:
        cur_res = np.dot(cur_res, layer)
        if activation_func:
            cur_res = activation_func(cur_res)
        hidden_layer_res_buf.append(cur_res)
    return cur_res

In [ ]:
# back propagation to update model
def back_propagation(td_err):
    # TODO
    

In [ ]:
step_count_list = []
for ep in episodes:
    end = False
    step_count = 0
    
    # startup state
    new_state = preprocess_state(env.reset())
    
    # run until game end
    while end:
        # predict with the latest model
        state = new_state
        q_values = model_forward(state)
        
        # epsilon-greedy action selection
        if np.random().rand(1) > epsilon:
            action = np.argmax(q_values)
        else:
            action = env.action_space.sample()

        # step forward
        new_state_no, reward, end, _ = env.step(action)
        step_count += 1
        
        # update q values with actual returns
        # save new state for the next step
        new_state = preprocess_state(new_state_no)
        
        # calculate error for back propagtion with Bellman equation
        td_err = reward + discount * np.max(model_forward(new_state)) - q_values[action]
        
        # back propagation with td error to update model
        back_propagation(td_err)
        
    # update epsilon
    epsilon = max(epsilon*epsilon_decay, epsilon_min)
    
    # record step counts
    step_count_list.append(step_count)
    # print informations
    # TODO
    print('In episode {}, {} steps are used.'.format(ep, step_count))
    

run with model

In [ ]:
end = False
state = preprocess_state(env.reset())
env.render()
while end:
    q_values = model_forward(state)
    action = np.argmax(q_values)
    state_no, _, end, _ = env.step(action)
    env.render()
    state = preprocess_state(state_no)